In [1]:
#!pip install nest_asyncio

In [2]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


#!pip install nest_asyncio


# In[66]:


import wave
import time
from datetime import datetime
import concurrent.futures
from dotenv import load_dotenv
from datetime import datetime
import os
import azure.cognitiveservices.speech as speechsdk
import uuid
import requests
import nest_asyncio
nest_asyncio.apply()


from azure.cognitiveservices.speech import AudioDataStream, SpeechConfig, SpeechRecognizer, AudioConfig
from azure.cognitiveservices.speech.audio import PushAudioInputStream, AudioStreamFormat
phrases_list = ["ई-एम-आई", "ब्याज दर", "तिरसठ", "इक्यावन", "उनचालीस", "सत्ताईस", "महीने", "उनतालीस"]


load_dotenv(override=True)


speech_ep = os.getenv('SPEECH_EP')
speech_key = os.getenv('SPEECH_KEY')

speech_region = os.getenv('SPEECH_REGION')
audiofile = "trimmed_16k_41.wav"


# In[ ]:





# In[ ]:





# In[67]:


def speech_recognize_continuous_from_file(audiofile, lang):

    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=speech_region)
    speech_config.output_format = speechsdk.OutputFormat.Detailed
    speech_config.speech_recognition_language=lang
    audio_config = speechsdk.audio.AudioConfig(filename=audiofile)

    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)
    
    
    done = False

    def stop_cb(evt: speechsdk.SessionEventArgs):
        """callback that signals to stop continuous recognition upon receiving an event `evt`"""
        nonlocal done
        done = True

    speech_recognizer.recognized.connect(lambda evt: print('RECOGNIZED: {}'.format(evt)))
    speech_recognizer.session_stopped.connect(stop_cb)
    speech_recognizer.canceled.connect(stop_cb)

    speech_recognizer.start_continuous_recognition_async()
    while not done:
        time.sleep(.001)

    speech_recognizer.stop_continuous_recognition_async()
   


# In[68]:


start_time = datetime.now()
result = speech_recognize_continuous_from_file("trimmed_16k_41.wav", "hi-IN")
end_time = datetime.now()
print(end_time-start_time)


# In[ ]:





# In[69]:


def trans_with_api_code(audio_file_path, language):
    endpoint = f"https://{speech_region}.stt.speech.microsoft.com/speech/recognition/conversation/cognitiveservices/v1?language={language}&format=detailed"

    session_id = str(uuid.uuid4()).replace('-', '')

    headers = {
        'Ocp-Apim-Subscription-Key': speech_key,
        'Content-Type': 'audio/wav; samplerate=8000',
        'Accept': 'application/json',
        'X-ConnectionId': session_id
    }

    try:
        with open(audio_file_path, "rb") as audio_file:
            audio_data = audio_file.read()

        request_body = {
            "recognitionParameters": {
                "wordLevelConfidence": True,
                "profanityFilterMode": "Masked",
                "grammar": {
                    "externalPhrases": phrases_list
                }
            }

        }

        start_time = datetime.now()
        response = requests.post(endpoint, headers=headers, data=audio_data, json=request_body)

    
        if response.status_code == 200:
            result = response.json()
            
            print(f"result:{result}")
        else:
            print("error", response.json())
            

    except Exception as e:
        print(f"An error occurred: {e}")


# In[70]:


start_time = datetime.now()
result = trans_with_api_code("trimmed_16k_41.wav", "hi-IN")
end_time = datetime.now()
print(end_time-start_time)


# In[50]:


# Measuring latency: https://learn.microsoft.com/en-us/answers/questions/752263/azure-speech-to-text-how-can-sdk-client-know-the-l


# In[75]:









RECOGNIZED: SpeechRecognitionEventArgs(session_id=23b4f267249f4c199172833a796c2c2a, result=SpeechRecognitionResult(result_id=c5991f8e1b5b4f0694d1791f2f1c5bda, text="नमस्ते मेन पूजा बजाज फनैन्स के आपकी वर्चुअल असिस्टेंट आपको लोन का रेक्विरेमेंट है क्या?", reason=ResultReason.RecognizedSpeech))
0:00:02.210073
result:{'RecognitionStatus': 'Success', 'Offset': 1100000, 'Duration': 70000000, 'NBest': [{'Confidence': 0.74739945, 'Lexical': 'नमस्ते मेन पूजा बजाज फनैन्स के आपकी वर्चुअल असिस्टेंट आपको लोन का रेक्विरेमेंट है क्या', 'ITN': 'नमस्ते मेन पूजा बजाज फनैन्स के आपकी वर्चुअल असिस्टेंट आपको लोन का रेक्विरेमेंट है क्या', 'MaskedITN': 'नमस्ते मेन पूजा बजाज फनैन्स के आपकी वर्चुअल असिस्टेंट आपको लोन का रेक्विरेमेंट है क्या', 'Display': 'नमस्ते मेन पूजा बजाज फनैन्स के आपकी वर्चुअल असिस्टेंट आपको लोन का रेक्विरेमेंट है क्या?'}], 'DisplayText': 'नमस्ते मेन पूजा बजाज फनैन्स के आपकी वर्चुअल असिस्टेंट आपको लोन का रेक्विरेमेंट है क्या?'}
0:00:02.066031
SESSION STARTED: SessionEventArgs(session_id=0c

In [5]:
segments = []
def streaming_call_back(idx):
    global segments
    n_bytes = 6400 * 45
    if len(segments) == 0:
        file_path = "trimmed_16k_41.wav"
        with wave.open(file_path, 'rb') as wav_fh:
            while True:
                frames = wav_fh.readframes(n_bytes // wav_fh.getsampwidth())
                if not frames:
                    break
                segments.append(frames)
    if idx < len(segments):
        data = segments[idx]
    if idx == len(segments) - 1:
        next_idx = -1
    else:
        next_idx = idx + 1
    
    time.sleep(0.04)
    start_time1 = datetime.now() 
    return {"data":data, "next_idx":next_idx, "start_time": start_time1}

            
def trans_with_streaming(streaming_call_back, lang, start_time=None):
    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=speech_region)
    speech_config.output_format = speechsdk.OutputFormat.Detailed
    stream = speechsdk.audio.PushAudioInputStream()
    audio_config = speechsdk.audio.AudioConfig(stream=stream)
    speech_recognizer = speechsdk.SpeechRecognizer(language=lang, speech_config=speech_config, audio_config=audio_config)
    
    text = []
    done=False
    def handle_recognized(evt):
        #This will give you confidence score
        #print(evt.result.json)
        if evt.result.reason == speechsdk.ResultReason.RecognizedSpeech:
            
            #print(start_time, end_time)
            try:
                latency = evt.result.properties[speechsdk.PropertyId.SpeechServiceResponse_RecognitionLatencyMs]
        
                print(f"Recognition latency: {latency} ms")
            except Exception as e:
                print(e)
            text.append(evt.result.text)
            print(evt.result.text)
            end_time = datetime.now()
            print(end_time-start_time)
            nonlocal done
            done = True
        elif evt.result.reason == speechsdk.ResultReason.NoMatch:
            print('NOMATCH: Speech could not be recognized.')

    def handle_canceled(evt):
        print('CANCELED: {}'.format(evt))
        if evt.reason == speechsdk.CancellationReason.Error:
            print('CANCELED: Error details - {}'.format(evt.error_details))
        speech_recognizer.stop_continuous_recognition_async()
    
    
    speech_recognizer.recognized.connect(handle_recognized)
    speech_recognizer.session_started.connect(lambda evt: print('SESSION STARTED: {}'.format(evt)))
    speech_recognizer.session_stopped.connect(lambda evt: print('SESSION STOPPED: {}'.format(evt)))
    speech_recognizer.canceled.connect(handle_canceled)
    

    next_idx = 0
    try:
        speech_recognizer.start_continuous_recognition_async()
        while next_idx >= 0:
            resp = streaming_call_back(next_idx)
            stream.write(resp["data"])
            next_idx = resp["next_idx"]
            start_time1 = resp["start_time"]
            #print(start_time)
            #time.sleep(0.01)
        
            #stream.close()
    except AssertionError as e:
        print(f"Audio format error: {e}")
    except Exception as e:
        print(f"Error reading audio file: {e}")
    finally:
        print("Done")
        while not done:
            time.sleep(0.001)
        speech_recognizer.stop_continuous_recognition_async()

    return ' '.join(text)


# In[76]:


start_time = datetime.now()
trans_with_streaming(streaming_call_back, "hi-IN",start_time)
end_time = datetime.now()
print(f"Time taken as measured from calling function: {end_time-start_time}")


# In[ ]:

SESSION STARTED: SessionEventArgs(session_id=c4532a9686604d81942b80d753e46626)
Done
Recognition latency: 2261 ms
नमस्ते मेन पूजा बजाज फनैन्स के आपकी वर्चुअल असिस्टेंट आपको लोन का रेक्विरेमेंट है क्या?
0:00:03.363288
Time taken as measured from calling function: 0:00:03.379048
SESSION STOPPED: SessionEventArgs(session_id=c4532a9686604d81942b80d753e46626)
